In [20]:
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json
from io import StringIO
import pygwalker as pyg
import os
from tqdm import tqdm
import polars.selectors as cs
from scipy.stats import zscore 

reading competition editions json

In [2]:
ce_json_path = './skillcorner-competition-editions.json'
ce_df1 = pl.read_json(ce_json_path)
ce_df1.head(1)
#ce_df.columns

count,next,previous,results
i64,null,null,list[struct[4]]
938,null,null,"[{0,{0,""FIFA"",""World Cup"",""male"",""adult""},{10,2014,2014,""2014""},""FIFA - World Cup - 2014""}, {1,{1,""ENG"",""Premier League"",""male"",""adult""},{2,2015,2016,""2015/2016""},""ENG - Premier League - 2015/2016""}, … {1129,{221,""INT"",""Pinatar National Team Friendlies"",""male"",""U21""},{128,2025,2025,""2025""},""INT - Pinatar National Team Friendlies - 2025""}]"


In [3]:
#ce_df2 = ce_df1.explode('results')
ce_df2 = ce_df1.select('results').explode('results')
ce_df2 = ce_df2.unnest('results')
ce_df2.head(1)

id,competition,season,name
i64,struct[5],struct[4],str
0,"{0,""FIFA"",""World Cup"",""male"",""adult""}","{10,2014,2014,""2014""}","""FIFA - World Cup - 2014"""


In [4]:
ce_name_id = ce_df2.select([
    pl.col('name'),
    pl.col('id'),
    ]
)
ce_name_id.head(1)

name,id
str,i64
"""FIFA - World Cup - 2014""",0


In [5]:
ce_competition = ce_df2.select('competition').unnest('competition')
ce_competition = ce_competition.rename({col: f"competition_{col}" for col in ce_competition.columns})
ce_competition.head(1)

competition_id,competition_area,competition_name,competition_gender,competition_age_group
i64,str,str,str,str
0,"""FIFA""","""World Cup""","""male""","""adult"""


In [6]:
ce_season = ce_df2.select('season').unnest('season')
ce_season = ce_season.rename({col: f"season_{col}" for col in ce_season.columns})
ce_season.head(1)

season_id,season_start_year,season_end_year,season_name
i64,i64,i64,str
10,2014,2014,"""2014"""


In [7]:
ce = pl.concat([
    ce_name_id,
    ce_season,
    ce_competition
],
how = "horizontal",)
ce.head(10)

name,id,season_id,season_start_year,season_end_year,season_name,competition_id,competition_area,competition_name,competition_gender,competition_age_group
str,i64,i64,i64,i64,str,i64,str,str,str,str
"""FIFA - World Cup - 2014""",0,10,2014,2014,"""2014""",0,"""FIFA""","""World Cup""","""male""","""adult"""
"""ENG - Premier League - 2015/20…",1,2,2015,2016,"""2015/2016""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""UEFA - European Championship -…",2,12,2016,2016,"""2016""",2,"""UEFA""","""European Championship""","""male""","""adult"""
"""ENG - Premier League - 2016/20…",3,3,2016,2017,"""2016/2017""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""ENG - Premier League - 2017/20…",4,4,2017,2018,"""2017/2018""",1,"""ENG""","""Premier League""","""male""","""adult"""
"""FRA - Ligue 1 - 2016/2017""",5,3,2016,2017,"""2016/2017""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""FRA - Ligue 1 - 2017/2018""",6,4,2017,2018,"""2017/2018""",3,"""FRA""","""Ligue 1""","""male""","""adult"""
"""ESP - LaLiga - 2016/2017""",7,3,2016,2017,"""2016/2017""",4,"""ESP""","""LaLiga""","""male""","""adult"""
"""ESP - LaLiga - 2017/2018""",8,4,2017,2018,"""2017/2018""",4,"""ESP""","""LaLiga""","""male""","""adult"""


reading json player file- test on 1 file


In [8]:
position_json_testpath = './skillcorner-20250214/metrics-ce249-rAM.json'

position_test_df = pl.read_json(position_json_testpath)

In [9]:
position_test_df.columns
#position_test_df.head(10)

['Count HSR 2',
 'HSR Distance 1',
 'Count Medium Deceleration TIP 1',
 'Sprinting Distance OTIP 2',
 'Sprinting Distance TIP',
 'Minutes OTIP',
 'Count High Acceleration TIP 1',
 'HSR Distance OTIP',
 'Sprinting Distance 2',
 'Count Medium Acceleration OTIP',
 'Distance TIP 1',
 'HSR Distance TIP 1',
 'group',
 'Count High Deceleration OTIP',
 'Count High Acceleration 1',
 'match_id',
 'Sprinting Distance OTIP 1',
 'HSR Distance OTIP 2',
 'season_id',
 'Minutes 1',
 'Count High Deceleration 1',
 'Count High Acceleration OTIP 2',
 'Count High Deceleration OTIP 2',
 'Count High Acceleration TIP 2',
 'competition_edition_id',
 'Running Distance OTIP 1',
 'HSR Distance TIP 2',
 'Count Medium Deceleration OTIP 2',
 'quality_check',
 'position',
 'Count Medium Acceleration',
 'team_name',
 'Count Sprint TIP 1',
 'Count Sprint',
 'Minutes OTIP 2',
 'Running Distance 1',
 'HSR Distance OTIP 1',
 'Count High Deceleration TIP',
 'Sprinting Distance',
 'Sprinting Distance 1',
 'Distance OTIP 1',

In [10]:
#define the path to the entire directory
json_dir_path = './skillcorner-20250214/'

### Look at correlations between parameters
Looking at the correlations across all competitions across all positions to see if we can leave out certain parameters

In [11]:
json_list = os.listdir(json_dir_path)
#json_list

In [12]:
"""dfs = []
for jsonf in json_list:
    json_path = f"{json_dir_path}{jsonf}"
    dfs.append(pl.read_json(json_path))

corr_df = pl.concat(dfs, how="align_full")"""

'dfs = []\nfor jsonf in json_list:\n    json_path = f"{json_dir_path}{jsonf}"\n    dfs.append(pl.read_json(json_path))\n\ncorr_df = pl.concat(dfs, how="align_full")'

In [13]:
#loop through the json files and concat them into one big df

# creating an empty data frame to concat the json dataframes to
read_json_list = []

for jsonf in tqdm(json_list):
    json_path = f"{json_dir_path}{jsonf}"
    json_df = pl.read_json(json_path)

    read_json_list.append(json_df)

corr_df = pl.concat(read_json_list)


corr_df.shape
#corr_df.head(2)
"""
output_path = "json_output.csv"
corr_df.write_csv(output_path)"""

  0%|          | 0/1652 [00:00<?, ?it/s]

100%|██████████| 1652/1652 [02:06<00:00, 13.06it/s]


'\noutput_path = "json_output.csv"\ncorr_df.write_csv(output_path)'

In [19]:
nonull_numeric = corr_df.drop_nulls().select(cs.numeric())

In [ ]:
correlations = nonull_numeric.corr()

In [ ]:
# only retaining meaningfull correlation

# Initialize an empty list to store the filtered correlation pairs
filtered_data = []

# Iterate over the rows and columns to extract variable pairs and correlations
for i, row in enumerate(correlations.columns):
    for j, col in enumerate(correlations.columns):
        if i < j:  # Skip self-correlation and duplicate pairs (ensure we only process one pair)
            corr_value = correlations[row][j]  # Access the correlation value at row i, column j
            if corr_value > 0.8:  # Only keep correlations > 0.8
                filtered_data.append([row, col, corr_value])

# Create a Polars DataFrame with the filtered data
filtered_df = pl.DataFrame(filtered_data, schema=["Variable 1", "Variable 2", "Correlation"])

# Write the filtered data to a CSV file (since Polars doesn't support writing to Excel directly)
filtered_df.write_excel(workbook = "filtered_correlations.xlsx")

print("Filtered correlations saved to 'filtered_correlations.csv'.")

In [22]:
nonull_numeric_np = nonull_numeric.to_numpy()

In [23]:
#get back the correlations
#get back the z-scores
z_scores = zscore(nonull_numeric_np)

In [28]:
# Find outliers
outliers = nonull_numeric_np[(np.abs(zscore(nonull_numeric_np)) >= 5).all(axis=1)]

# Check the shape
print(outliers.shape)

(0, 106)


In [10]:
pyg.walk(position_test_df)

Box(children=(HTML(value='\n<div id="ifr-pyg-0006324367bf2c7fVw7Zdze1N3WBjGS2" style="height: auto">\n    <hea…

ERROR: kanaries_api_key is not valid.
Please set kanaries_api_key first.
If you are not kanaries user, please register it from 'https://kanaries.net/home/access' 
Then refer 'https://github.com/Kanaries/pygwalker/wiki/How-to-get-api-key-of-kanaries%3F' to set kanaries_api_key. 

